In [ ]:
import sys
import os

parent_path = os.path.abspath("..")  # Get absolute path of parent directory
if parent_path not in sys.path:  
    sys.path.append(parent_path)  # Add only if it's not already there
    
from dotenv import load_dotenv
load_dotenv("../.env")


import pandas as pd
import chromadb
import openai

from src.models.chromadb_functions import *  
from src.models.gpt_models import  correct_text_with_gpt

### Read Excel file


In [13]:
question_answer_df = pd.read_excel("../data/question_answer/QA.xlsx")

print(question_answer_df)

print("Excel Length: ", len(question_answer_df))

   no                    base_question                 answer  \
0   1   Tại sao cần bón phân cho cây ?        Để cây tươi tốt   
1   2  Cây nhãn chăm sóc như thế nào ?  Bón phân thường xuyên   

                                    variant_question  
0  Lí do cần bón phân cho cây \nTại sao cây cần p...  
1  Chăm sóc cây nhãn ra sao ?\nLàm sao chăm sóc c...  
Excel Length:  2


### Get ChromaDB Collection

In [14]:
collection_path     = "../data/vector_database"
collection_name     = os.getenv("CHROMADB_COLLECTION_NAME")
embedding_func      = define_embedding_function( api_key = os.getenv("OPENAI_API_KEY"), model_name = os.getenv("EMBEDDING_MODEL"))
similarity_method   = os.getenv("SIMILARITY_METHOD")

chromadb_collection = get_chroma_collection(collection_path, collection_name, embedding_func, similarity_method)

### Loop through variant question

In [ ]:
total_count = 0

for index, row in question_answer_df.iterrows():
    question_index  = row["no"]
    base_question   = row["base_question"]
    variant_question_list   = row["variant_question"].split("\n")
    
    print("Process question: ", base_question)
    
    match_count = 0
    
    for quest in variant_question_list:
        
        print("    Variant question: ", quest)
        # Fix question first
        fixed_quest = correct_text_with_gpt(input_text = quest)
        
        results = chromadb_collection.query(
            query_texts=[fixed_quest], # Chroma will embed this for you
            n_results=1 # how many results to return
        )
        
        if results["documents"][0][0] == base_question:
            match_count +=1
            print("        Passed")
        else:
            print("        Failed")
    
    print("\n")
    
    accuracy = match_count/len(variant_question_list)
    if accuracy >= 0.8:
        total_count += 1
        print("  Accuracy pass: ", accuracy)
    else:
        print("  Accuracy failed: ", accuracy)
    
    print("----------------------------")
        
        
print("Total Summary: ", total_count/len(question_answer_df))
    

Process question:  Tại sao cần bón phân cho cây ?
    Variant question:  Lí do cần bón phân cho cây 
        Passed
    Variant question:  Tại sao cây cần phân bón ?
        Passed
    Variant question:  Taij saoo bons phân cho caay
        Passed
  Accuracy pass:  1.0
----------------------------
Process question:  Cây nhãn chăm sóc như thế nào ?
    Variant question:  Chăm sóc cây nhãn ra sao ?
        Passed
    Variant question:  Làm sao chăm sóc cây nhãn cho tốt 
        Passed
    Variant question:  Khi chăm cây nhãn caanff phải làm chi 
        Passed
  Accuracy pass:  1.0
----------------------------
Total Summary:  1.0
